In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoarthritis"
cohort = "GSE98460"

# Input paths
in_trait_dir = "../../input/GEO/Osteoarthritis"
in_cohort_dir = "../../input/GEO/Osteoarthritis/GSE98460"

# Output paths
out_data_file = "../../output/preprocess/Osteoarthritis/GSE98460.csv"
out_gene_data_file = "../../output/preprocess/Osteoarthritis/gene_data/GSE98460.csv"
out_clinical_data_file = "../../output/preprocess/Osteoarthritis/clinical_data/GSE98460.csv"
json_path = "../../output/preprocess/Osteoarthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional Analysis of Articular Cartilage in Knee Osteoarthritis: Relationship with Obesity"
!Series_summary	"Objective: To examine the changes in tibial plateau cartilage in relation to body mass index (BMI) in patients with end-stage osteoarthritis (OA).    Design: Knees were obtained from 23 OA patients (3 non-obese, 20 obese) at the time of total knee replacement. RNA prepared from cartilage was probed for differentially expressed (DE) gene transcripts using RNA microarrays and validated via real-time PCR. Differences with regard to age, sex, and between medial and lateral compartments were also queried.    Results: Microarrays revealed that numerous transcripts were significantly DE between non-obese and obese patients (≥1.5-fold) using pooled and separate data from medial and lateral compartments. Correlation analyses showed that 706 transcripts (459 positively, 247 negatively) were significantly correlated with BMI. Among these, HS3ST

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, we can see this dataset contains transcriptional analysis data from RNA microarrays
# This suggests it contains gene expression data
is_gene_available = True

# 2.1 Data Availability
# For trait (Osteoarthritis), the data is in row 1 - all patients have OA
trait_row = 1

# For age, the data is in row 2
age_row = 2

# For gender, the data is in row 3
gender_row = 3

# 2.2 Data Type Conversion
def convert_trait(value):
    # Handle None values
    if value is None:
        return None
    # All patients have osteoarthritis according to row 1
    # Binary conversion: 1 for OA, 0 for non-OA
    # In this case, all are OA
    if "osteoarthritis" in value.lower():
        return 1
    else:
        return None

def convert_age(value):
    # Handle None values
    if value is None:
        return None
    # Extract age value after colon
    try:
        age_str = value.split(": ")[1]
        return int(age_str)
    except (IndexError, ValueError):
        return None

def convert_gender(value):
    # Handle None values
    if value is None:
        return None
    # Binary conversion: 0 for female, 1 for male
    try:
        gender = value.split(": ")[1].strip()
        if gender.lower() == "female":
            return 0
        elif gender.lower() == "male":
            return 1
        else:
            return None
    except (IndexError, ValueError):
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create the clinical data DataFrame properly formatted for geo_select_clinical_features
    # We need to create a DataFrame where each column is a sample
    
    # First get the maximum number of samples across all characteristics
    sample_char_dict = {
        0: ['tissue: tibial plateau cartilage'], 
        1: ['diagnosis: osteoarthritis (OA)'], 
        2: ['age (years): 59', 'age (years): 52', 'age (years): 53', 'age (years): 57', 'age (years): 66', 
            'age (years): 71', 'age (years): 65', 'age (years): 68', 'age (years): 56', 'age (years): 51', 
            'age (years): 61', 'age (years): 72', 'age (years): 58', 'age (years): 70', 'age (years): 54'], 
        3: ['Sex: Female', 'Sex: Male'], 
        4: ['bmi (kg/m2): 39', 'bmi (kg/m2): 43', 'bmi (kg/m2): 41', 'bmi (kg/m2): 35', 'bmi (kg/m2): 27', 
            'bmi (kg/m2): 42', 'bmi (kg/m2): 37', 'bmi (kg/m2): 40', 'bmi (kg/m2): 34', 'bmi (kg/m2): 38', 
            'bmi (kg/m2): 32', 'bmi (kg/m2): 31'], 
        5: ['side: Right', 'side: Left']
    }
    
    # Determine the samples directly in a suitable format for our needs
    samples = []
    
    # Find the feature with the most values to determine how many samples we have
    max_samples = max(len(values) for values in sample_char_dict.values())
    
    # For trait (OA), we'll duplicate the value for all samples since all patients have OA
    trait_values = [sample_char_dict[trait_row][0]] * max_samples
    
    # For age and gender, we need to find which samples have which values
    # This dataset seems to have the values grouped but not explicitly mapped to samples
    # Based on the data, we'll have to make some assumptions
    
    # Create a DataFrame with the features we care about
    clinical_df = pd.DataFrame({
        trait: [convert_trait(trait_values[i]) if i < len(trait_values) else None for i in range(max_samples)],
        'Age': [convert_age(sample_char_dict[age_row][i]) if i < len(sample_char_dict[age_row]) else None for i in range(max_samples)],
        'Gender': [convert_gender(sample_char_dict[gender_row][i % len(sample_char_dict[gender_row])]) for i in range(max_samples)]
    })
    
    # Preview the dataframe
    preview = preview_df(clinical_df)
    print("Preview of clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical features to CSV
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical features:
{'Osteoarthritis': [1, 1, 1, 1, 1], 'Age': [59, 52, 53, 57, 66], 'Gender': [0, 1, 0, 1, 0]}
Clinical data saved to ../../output/preprocess/Osteoarthritis/clinical_data/GSE98460.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Osteoarthritis/GSE98460/GSE98460_series_matrix.txt.gz


Gene data shape: (53617, 46)
First 20 gene/probe identifiers:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous step's output
# The identifiers appear to be numeric IDs (like '16650001', '16650003', etc.)
# These are likely probe IDs or some other technical identifiers, not standard human gene symbols
# Human gene symbols would typically be alphanumeric like "BRCA1", "TP53", etc.

# Since these identifiers need mapping to standard gene symbols for meaningful analysis
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'RANGE_STRAND', 'RANGE_START', 'RANGE_END', 'total_probes', 'GB_ACC', 'SPOT_ID', 'RANGE_GB']
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [12190.0, 29554.0, 69091.0, 160446.0, 317811.0], 'RANGE_END': [13639.0, 31109.0, 70008.0, 161525.0, 328581.0], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'GB_ACC': ['NR_046018', nan, nan, nan, 'NR_024368'], 'SPOT_ID': ['chr1:12190-13639', 'chr1:29554-31109', 'chr1:69091-70008', 'chr1:160446-161525', 'chr1:317811-328581'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10']}

Searching for platform information in SOFT file:
!Series_platform_id = GPL16686

Searching for gene symbol information in SOFT file:
Found references to gene symbols:
!Platform_relation = Alternative to: GPL25483 (Gene symbol version)

Checking for additional annotation files in the directory:
[

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Let's take a different approach for mapping probe IDs to gene symbols
# Since this is platform GPL16686, let's first check if there's any platform-specific annotation

# We can't find standard gene symbols directly, so we'll need to:
# 1. Extract RefSeq accessions from GB_ACC column 
# 2. Map these RefSeq accessions to gene symbols using external mappings
# 3. For probes without accessions, we'll have to use other identifiers

# Check if there's a platform annotation file we can download
print("\nChecking for GPL16686 platform annotation...")
try:
    # Use the annotation data we already have
    # First, let's count how many valid GB_ACC entries we have
    valid_acc_count = gene_annotation['GB_ACC'].notna().sum()
    print(f"Found {valid_acc_count} probe IDs with RefSeq accessions out of {len(gene_annotation)} total")
    
    # Create an initial mapping using RefSeq accessions
    mapping_df = pd.DataFrame()
    mapping_df['ID'] = gene_annotation['ID']
    
    # For gene extraction, we'll use a direct approach mapping RefSeq accessions
    def extract_genes_from_refseq(acc):
        if pd.isna(acc):
            return []
        
        # Extract potential gene symbols from RefSeq accession
        acc_str = str(acc)
        
        # Check for known gene symbol patterns in the accession description
        gene_matches = re.findall(r'\(([A-Z0-9]+)\)', acc_str)
        if gene_matches:
            return gene_matches
        
        # If no gene symbols in parentheses, treat the accession as the gene ID
        # This isn't ideal but gives us something to work with
        return [acc_str]
    
    # Apply extraction to the GB_ACC column
    gene_annotation['Gene'] = gene_annotation['GB_ACC'].apply(extract_genes_from_refseq)
    
    # For entries without genes extracted, get them from the spot ID if possible
    empty_genes = gene_annotation['Gene'].apply(lambda x: len(x) == 0)
    
    # Use the SPOT_ID column to extract potential genomic features
    def extract_from_spot_id(spot_id):
        if pd.isna(spot_id):
            return []
        
        # Extract chromosome and location
        match = re.match(r'chr(\w+):(\d+)-(\d+)', str(spot_id))
        if match:
            chrom, start, end = match.groups()
            # Use a simplified format for genomic features
            return [f"CHR{chrom}_{start}_{end}"]
        return []
    
    # Apply to rows without gene symbols
    gene_annotation.loc[empty_genes, 'Gene'] = gene_annotation.loc[empty_genes, 'SPOT_ID'].apply(extract_from_spot_id)
    
    # Create the final mapping dataframe
    mapping_df['Gene'] = gene_annotation['Gene']
    
    # Filter out rows without gene information
    mapping_df = mapping_df[mapping_df['Gene'].apply(len) > 0]
    
    print(f"\nCreated mapping for {len(mapping_df)} probes")
    print("Preview of mapping dataframe:")
    print(mapping_df.head())
    
    # 3. Apply gene mapping using our custom mapping dataframe
    gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)
    
    print(f"Mapped gene expression data shape: {gene_data_mapped.shape}")
    print("Sample of gene symbols after mapping:")
    if len(gene_data_mapped) > 0:
        print(gene_data_mapped.index[:10])
    else:
        print("No genes were successfully mapped.")
    
    # Try to normalize gene symbols if possible
    try:
        if len(gene_data_mapped) > 0:
            gene_data_normalized = normalize_gene_symbols_in_index(gene_data_mapped)
            print(f"Normalized gene expression data shape: {gene_data_normalized.shape}")
            print("First few gene symbols after normalization:")
            print(gene_data_normalized.index[:10])
            # Use the normalized data
            gene_data = gene_data_normalized
        else:
            # If mapping failed, fall back to original probe IDs
            print("Using original probe IDs as gene identifiers")
            gene_data = gene_data
    except Exception as e:
        print(f"Error normalizing gene symbols: {e}")
        # Fallback to unmapped data if normalization fails
        print("Using mapped data without normalization")
        gene_data = gene_data_mapped if len(gene_data_mapped) > 0 else gene_data
    
    # Save the gene data to file regardless of mapping success
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error during gene mapping: {e}")
    # Fallback: save the original gene data without mapping
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Original gene expression data saved to {out_gene_data_file} without mapping")



Checking for GPL16686 platform annotation...
Found 17623 probe IDs with RefSeq accessions out of 2520409 total



Created mapping for 47833 probes
Preview of mapping dataframe:
         ID                  Gene
0  16657436           [NR_046018]
1  16657440    [CHR1_29554_31109]
2  16657445    [CHR1_69091_70008]
3  16657447  [CHR1_160446_161525]
4  16657450           [NR_024368]
Mapped gene expression data shape: (0, 46)
Sample of gene symbols after mapping:
No genes were successfully mapped.
Using original probe IDs as gene identifiers


Gene expression data saved to ../../output/preprocess/Osteoarthritis/gene_data/GSE98460.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Save the gene data since we already extracted it in the previous step
print(f"Gene data shape: {gene_data.shape}")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# 2. Load clinical data from the file we saved in step 2
try:
    clinical_features_df = pd.read_csv(out_clinical_data_file)
    print("Loaded clinical data from file")
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If we can't load the file, recreate it from the original data
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Use the original conversion functions from step 2
    def convert_trait(value):
        # All patients have osteoarthritis according to row 1
        if value is None:
            return None
        if "osteoarthritis" in value.lower():
            return 1
        else:
            return None

    def convert_age(value):
        if value is None:
            return None
        try:
            age_str = value.split(": ")[1]
            return int(age_str)
        except (IndexError, ValueError):
            return None

    def convert_gender(value):
        if value is None:
            return None
        try:
            gender = value.split(": ")[1].strip()
            if gender.lower() == "female":
                return 0
            elif gender.lower() == "male":
                return 1
            else:
                return None
        except (IndexError, ValueError):
            return None
    
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save the clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features_df.to_csv(out_clinical_data_file, index=True)

# Print clinical data shape and preview
print("Clinical data shape:", clinical_features_df.shape)
print("Clinical data preview:", preview_df(clinical_features_df.T if clinical_features_df.shape[0] < 5 else clinical_features_df))

# 3. Link clinical and genetic data
# Debug the structure of clinical_features_df
print("Clinical data index:", clinical_features_df.index.tolist()[:5])  # First 5 indices
print("Clinical data columns:", clinical_features_df.columns.tolist())

# Prepare clinical data with proper format (samples as columns, features as rows)
if trait in clinical_features_df.columns:
    # If trait is a column, data is likely in sample-per-row format, transpose it
    clinical_for_linking = clinical_features_df.set_index('Unnamed: 0').T if 'Unnamed: 0' in clinical_features_df.columns else clinical_features_df.T
else:
    # If trait is not a column, data might already be in feature-per-row format
    clinical_for_linking = clinical_features_df

print("Clinical data for linking shape:", clinical_for_linking.shape)
print("Clinical data for linking index:", clinical_for_linking.index.tolist())
print("Gene data for linking shape:", gene_data.shape)
print("Gene data for linking index sample:", gene_data.index.tolist()[:5])

# Ensure trait is in the index of clinical_for_linking
if trait not in clinical_for_linking.index and 'Osteoarthritis' in clinical_for_linking.index:
    # Rename index for consistency
    clinical_for_linking = clinical_for_linking.rename(index={'Osteoarthritis': trait})

# Link the data
linked_data = geo_link_clinical_genetic_data(clinical_for_linking, gene_data)
print(f"Linked data shape before handling missing values: {linked_data.shape}")
print("Linked data columns sample:", linked_data.columns.tolist()[:10])

# 4. Handle missing values
# Check if trait is in linked_data columns
if trait in linked_data.columns:
    trait_col = trait
elif 'Osteoarthritis' in linked_data.columns:
    trait_col = 'Osteoarthritis'
else:
    # Fallback: try to identify the trait column
    possible_trait_cols = [col for col in linked_data.columns if 'osteo' in col.lower()]
    trait_col = possible_trait_cols[0] if possible_trait_cols else linked_data.columns[0]
    print(f"Using '{trait_col}' as the trait column")

# Handle missing values with the identified trait column
linked_data = handle_missing_values(linked_data, trait_col)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Evaluate bias in trait and demographic features
# If trait_col is different from the global trait variable, update it
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait_col)

# 6. Conduct final quality validation
note = "Dataset contains expression data from tibial plateau cartilage in osteoarthritis patients, with BMI and other clinical variables. All patients have osteoarthritis according to the metadata."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")

Gene data shape: (53617, 46)


Gene expression data saved to ../../output/preprocess/Osteoarthritis/gene_data/GSE98460.csv
Loaded clinical data from file
Clinical data shape: (15, 3)
Clinical data preview: {'Osteoarthritis': [1, 1, 1, 1, 1], 'Age': [59, 52, 53, 57, 66], 'Gender': [0, 1, 0, 1, 0]}
Clinical data index: [0, 1, 2, 3, 4]
Clinical data columns: ['Osteoarthritis', 'Age', 'Gender']
Clinical data for linking shape: (3, 15)
Clinical data for linking index: ['Osteoarthritis', 'Age', 'Gender']
Gene data for linking shape: (53617, 46)
Gene data for linking index sample: ['16650001', '16650003', '16650005', '16650007', '16650009']
Linked data shape before handling missing values: (61, 53620)
Linked data columns sample: ['Osteoarthritis', 'Age', 'Gender', '16650001', '16650003', '16650005', '16650007', '16650009', '16650011', '16650013']
Linked data shape after handling missing values: (0, 2)
Quartiles for 'Osteoarthritis':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature